In [ ]:
RUN_ON_SAMPLE = True

In [1]:
import pandas as pd
import numpy as np
import os, sys, datetime, pickle, gc
from time import time, ctime
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
seeded_value = 88888
pd.set_option('display.max_colwidth', 50)
np.random.seed(seeded_value)

In [3]:
# suppress scientific notation
np.set_printoptions(suppress=True)

In [4]:
start_time = ctime(time())
print(start_time)

Tue Jun  9 00:36:45 2020


### Files
1. calendar.csv - Contains information about the dates on which the products are sold.
2. sales_train_validation.csv - Contains the historical daily unit sales data per product and store [d_1 - d_1913]
3. sample_submission.csv - The correct format for submissions. Reference the Evaluation tab for more info.
4. sell_prices.csv - Contains information about the price of the products sold per store and date.
5. sales_train_evaluation.csv - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)

In [5]:
DATA_DIR = "../data/"
RESULTS_DIR = "../results/"
PICKLE_DIR = "../data/preprocessed/"

In [6]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [7]:
def read_data():
    print('Reading files...')
    
    calendar = pd.read_csv('../data/calendar.csv').fillna("None")
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    
    sell_prices = pd.read_csv('../data/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    
    sales = pd.read_csv('../data/sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales.shape[0], sales.shape[1]))
    
    submission = pd.read_csv('../data/sample_submission.csv')
    
    pprint({
        "calendar.shape" : calendar.shape,
        "sell_prices.shape" : sell_prices.shape,
        "sales.shape" : sales.shape,
        "submission.shape" : submission.shape
    })
    
    return calendar, sell_prices, sales, submission

In [8]:
def add_dateparts(calendar, datecolname):
    calendar["Date"] = pd.to_datetime(calendar[datecolname], format = ("%Y-%m-%d"))
    calendar["Year"] = calendar["Date"].dt.year
    calendar["Quarter"] = calendar["Date"].dt.quarter
    calendar["Month"] = calendar["Date"].dt.month
    calendar["Week"] = calendar["Date"].dt.week
    calendar["Day"] = calendar["Date"].dt.day
    calendar["DOW"] = calendar["Date"].dt.dayofweek
    return calendar

In [10]:
def melt_join_fill(sales, calendar, sell_prices):
    if RUN_ON_SAMPLE:
        sales = pd.melt(sales.head(1_000),
                        id_vars=CARDINAL_VARS,
                        value_vars=Y_VARS,
                        var_name="day_id",
                        value_name='demand')
    else:
        sales = pd.melt(sales,
                        id_vars=CARDINAL_VARS,
                        value_vars=Y_VARS,
                        var_name="day_id",
                        value_name='demand')
    print("[INFO] ", "Melting  -- COMPLETE", ctime(time()))

    sales = pd.merge(sales, calendar, how="left", left_on="day_id", right_on="d")
    print("[INFO] ", "Merging1 -- COMPLETE", ctime(time()))
    
    sales = pd.merge(sales, sell_prices, how="left", on=["wm_yr_wk", "store_id", "item_id"])
    print("[INFO] ", "Merging2 -- COMPLETE", ctime(time()))
    
    # if sell_price is NA
    sales = sales.sort_values(by=CARDINAL_VARS+["date"])
    sales["sell_price_available"] = np.where(sales.sell_price.isna(),"N","Y")
    sales[["sell_price"]] = sales.groupby(["item_id"])[["sell_price"]].ffill()
    sales[["sell_price"]] = sales.groupby(["item_id"])[["sell_price"]].bfill()
    sales = sales.drop(["d", "wday", 'date', 'wm_yr_wk', 'weekday', 'month', 'year'], axis=1)
    
    print("[INFO] ", "Imputing -- COMPLETE", ctime(time()))    
    
    print("[INFO] ", "Final dataset contains", sales.shape)
    
    return sales.reset_index(drop=True)

In [11]:
if not os.path.exists(PICKLE_DIR+"merged_df.pickle"):
    CARDINAL_VARS = ['state_id', 'store_id', 'cat_id', 'dept_id', 'item_id']
    
    calendar, sell_prices, sales, submission = read_data()
    
    calendar = add_dateparts(calendar, "date")
    
    Y_VARS = sales.columns[sales.columns.str.startswith("d_")]

    data = melt_join_fill(sales, calendar, sell_prices)
    
    del sales, calendar, sell_prices
    gc.collect()

    print("Saving all data to ---> ", PICKLE_DIR)
    with open(os.path.join(PICKLE_DIR, "merged_df.pickle"),"wb") as f:
        pickle.dump((data), f)
else:
    print("Pickle exists hence loading from pickle file ---> ", PICKLE_DIR)
    with open(os.path.join(PICKLE_DIR+"merged_df.pickle"), "rb") as f:
        data = pickle.load(f)

Pickle exists hence loading from pickle file --->  ../data/preprocessed/


In [12]:
print(data.shape); data.head()

(1913000, 23)


,state_id,store_id,cat_id,dept_id,item_id,day_id,demand,event_name_1,event_type_1,event_name_2,...,snap_WI,Date,Year,Quarter,Month,Week,Day,DOW,sell_price,sell_price_available
0,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,d_1,0,None,None,None,...,0,2011-01-29,2011,1,1,4,29,5,9.578125,N
1,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,d_2,0,None,None,None,...,0,2011-01-30,2011,1,1,4,30,6,9.578125,N
2,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,d_3,0,None,None,None,...,0,2011-01-31,2011,1,1,5,31,0,9.578125,N
3,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,d_4,0,None,None,None,...,0,2011-02-01,2011,1,2,5,1,1,9.578125,N
4,CA,CA_1,HOBBIES,HOBBIES_1,HOBBIES_1_001,d_5,0,None,None,None,...,1,2011-02-02,2011,1,2,5,2,2,9.578125,N
